#### What We will Learn

1. S3 Buckets- Boto3
2. Iam Roles and Users
3. Complete Infrastructure of AWS Sagemaker-Training, Endpoints

In [57]:
import sagemaker
import sagemaker.session
from sklearn.model_selection import train_test_split
import boto3
import pandas as pd

sm_boto3 = boto3.client("sagemaker")
sess = sagemaker.Session()
region = sess.boto_session.region_name

bucket = 'mobilesagemakerbucket'
print("using bucket" + bucket)

using bucketmobilesagemakerbucket


In [3]:
print(region)

us-east-1


In [41]:
df = pd.read_csv('mob_price_classification_train.csv')
df.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,pc,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,2,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,6,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,6,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0,0,0,10,0.8,131,6,9,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0,13,1,44,0.6,141,2,14,1208,1212,1411,8,2,15,1,1,0,1


In [42]:
df.shape

(2000, 21)

In [43]:
df.isnull().sum()

battery_power    0
blue             0
clock_speed      0
dual_sim         0
fc               0
four_g           0
int_memory       0
m_dep            0
mobile_wt        0
n_cores          0
pc               0
px_height        0
px_width         0
ram              0
sc_h             0
sc_w             0
talk_time        0
three_g          0
touch_screen     0
wifi             0
price_range      0
dtype: int64

In [44]:
df['price_range'].value_counts()

price_range
1    500
2    500
3    500
0    500
Name: count, dtype: int64

In [45]:
features=list(df.columns)
features

['battery_power',
 'blue',
 'clock_speed',
 'dual_sim',
 'fc',
 'four_g',
 'int_memory',
 'm_dep',
 'mobile_wt',
 'n_cores',
 'pc',
 'px_height',
 'px_width',
 'ram',
 'sc_h',
 'sc_w',
 'talk_time',
 'three_g',
 'touch_screen',
 'wifi',
 'price_range']

In [47]:
label = features.pop(-1)
label

'price_range'

In [48]:
features

['battery_power',
 'blue',
 'clock_speed',
 'dual_sim',
 'fc',
 'four_g',
 'int_memory',
 'm_dep',
 'mobile_wt',
 'n_cores',
 'pc',
 'px_height',
 'px_width',
 'ram',
 'sc_h',
 'sc_w',
 'talk_time',
 'three_g',
 'touch_screen',
 'wifi']

In [49]:
x=df[features]
y=df[label]

In [50]:
X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.15, random_state=0)

In [51]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(1700, 20)
(300, 20)
(1700,)
(300,)


In [52]:
trainX = pd.DataFrame(X_train)
trainX[label] = y_train

testX = pd.DataFrame(X_test)
testX[label] = y_test

In [53]:
trainX

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,pc,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
1452,1450,0,2.1,0,1,0,31,0.6,114,5,4,1573,1639,794,11,5,9,0,1,1,1
1044,1218,1,2.8,1,3,0,39,0.8,150,7,14,1122,1746,1667,10,0,12,0,0,0,1
1279,1602,0,0.6,0,12,0,58,0.4,170,1,13,1259,1746,3622,17,2,17,0,1,1,3
674,1034,0,2.6,1,2,1,45,0.3,190,3,4,182,1293,969,15,1,7,1,0,0,0
1200,530,0,2.4,0,1,0,32,0.3,88,6,20,48,1012,959,17,7,6,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
835,1224,1,1.6,0,9,0,33,1.0,157,1,18,522,563,3796,10,5,13,1,1,0,3
1216,1158,0,0.7,1,1,1,29,0.7,123,2,20,311,1796,1542,17,9,15,1,0,1,1
1653,1190,0,2.0,1,0,0,40,0.2,93,5,1,1399,1646,3610,13,7,9,0,0,1,3
559,1191,0,2.4,1,2,0,13,0.9,169,1,7,179,1813,1028,14,6,8,1,1,1,0


In [54]:
testX

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,pc,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
405,1454,1,0.5,1,1,0,34,0.7,83,4,3,250,1033,3419,7,5,5,1,1,0,3
1190,1092,1,0.5,1,10,0,11,0.5,167,3,14,468,571,737,14,4,11,0,1,0,0
1132,1524,1,1.8,1,0,0,10,0.6,174,4,1,154,550,2678,16,5,13,1,0,1,2
731,1807,1,2.1,0,2,0,49,0.8,125,1,10,337,1384,1906,17,13,13,0,1,1,2
1754,1086,1,1.7,1,0,1,43,0.2,111,6,1,56,1150,3285,11,5,17,1,1,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
575,1197,1,0.5,1,9,0,20,0.1,129,5,10,214,1710,2885,19,7,14,1,1,1,2
982,1035,0,0.6,1,2,1,44,0.5,99,3,3,236,1960,3506,14,5,12,1,1,0,3
723,1271,0,0.5,1,2,0,54,0.1,159,8,3,613,650,990,14,11,6,0,1,1,0
1487,761,1,0.5,0,9,0,3,0.8,196,7,15,1325,1945,2248,19,3,15,1,1,1,2


In [55]:
trainX.to_csv("train-V-1.csv",index = False)
testX.to_csv("test-V-1.csv", index = False)

In [58]:
bucket

'mobilesagemakerbucket'

In [59]:
## send data to S3. Sagemaker will take the data for training from s3

sk_prefix = "sagemaker/mobile_price_sagemaker/sklearncontainer"
train_path = sess.upload_data(path='train-V-1.csv',bucket=bucket,key_prefix=sk_prefix)
test_path = sess.upload_data(path='test-V-1.csv',bucket=bucket,key_prefix=sk_prefix)

print(train_path)
print(test_path)

s3://mobilesagemakerbucket/sagemaker/mobile_price_sagemaker/sklearncontainer/train-V-1.csv
s3://mobilesagemakerbucket/sagemaker/mobile_price_sagemaker/sklearncontainer/test-V-1.csv


#### Script used by AWS Sagemaker To Train Models

In [61]:
%%writefile script.py

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix,precision_score
import sklearn
import joblib
import boto3
import pathlib
from io import StringIO
import argparse
import os
import numpy as np
import pandas as pd


def model_fn(model_dir):
    clf=joblib.load(os.path.join(model_dir,"model.joblib"))

if __name__=="__main__":

    print("[Info] Extracting arguments")
    parser=argparse.ArgumentParser()

    ## Hyperparameter
    parser.add_argument("--n_estimators",type=int,default=100)
    parser.add_argument("--random_state",type=int,default=0)

    ### Data,model,and output directories
    parser.add_argument("--model-dir", type=str, default=os.environ.get("SM_MODEL_DIR"))
    parser.add_argument("--train", type=str, default=os.environ.get("SM_CHANNEL_TRAIN"))
    parser.add_argument("--test", type=str, default=os.environ.get("SM_CHANNEL_TEST"))
    parser.add_argument("--train-file", type=str, default="train-V-1.csv")
    parser.add_argument("--test-file", type=str, default="test-V-1.csv")

    args, _ =parser.parse_known_args()

    print("SKLearn Version: ", sklearn.__version__)
    print("Joblib Version: ", joblib.__version__)

    print("[INFO] Reading data")
    print()
    train_df = pd.read_csv(os.path.join(args.train, args.train_file))
    test_df = pd.read_csv(os.path.join(args.test, args.test_file))

    features = list(train_df.columns)
    label = features.pop(-1)

    print("Building training and testing datasets")
    print()
    X_train = train_df[features]
    X_test = test_df[features]
    y_train = train_df[label]
    y_test = test_df[label]

    print('Column order: ')
    print(features)
    print()
    
    print("Label column is: ",label)
    print()
    
    print("Data Shape: ")
    print()
    print("---- SHAPE OF TRAINING DATA (85%) ----")
    print(X_train.shape)
    print(y_train.shape)
    print()
    print("---- SHAPE OF TESTING DATA (15%) ----")
    print(X_test.shape)
    print(y_test.shape)
    print()


    print("Training RandomForest Model ....")
    print()
    model=RandomForestClassifier(n_estimators=args.n_estimators,random_state=args.random_state,
                                 verbose=2,n_jobs=1)
    
    model.fit(X_train,y_train)

    print()

    model_path=os.path.join(args.model_dir,"model.joblib")
    joblib.dump(model,model_path)

    print("Model saved at" + model_path)

    y_pred_test = model.predict(X_test)
    test_acc = accuracy_score(y_test,y_pred_test)
    test_rep = classification_report(y_test,y_pred_test)

    print()
    print("---- METRICS RESULTS FOR TESTING DATA ----")
    print()
    print("Total Rows are: ", X_test.shape[0])
    print('[TESTING] Model Accuracy is: ', test_acc)
    print('[TESTING] Testing Report: ')
    print(test_rep)


Overwriting script.py


### AWS Sagemaker Entry Point To Execute the Training script

In [62]:
from sagemaker.sklearn.estimator import SKLearn

FRAMEWORK_VERSION="0.23-1"

sklearn_estimator=SKLearn(
    entry_point="script.py",
    role="arn:aws:iam::715763498984:role/sagemakeraccess",
    instance_count=1,
    instance_type="ml.m5.large",
    framework_version=FRAMEWORK_VERSION,
    base_job_name="RF-custom-sklearn",
    hyperparameters={
        "n_estimators":100,
        "random_state":0
    },
    use_spot_instance=True,
    max_run=3600

)

In [64]:
# launch training job, with asynchronous call
sklearn_estimator.fit({"train": train_path, "test": test_path}, wait=True)

[03/01/25 19:29:58] INFO     SageMaker Python SDK will collect telemetry to help us better  ]8;id=283915;file://s:\MLOPS\awssagemaker\venv\lib\site-packages\sagemaker\telemetry\telemetry_logging.py\telemetry_logging.py]8;;\:]8;id=781516;file://s:\MLOPS\awssagemaker\venv\lib\site-packages\sagemaker\telemetry\telemetry_logging.py#91\91]8;;\
                             understand our user's needs, diagnose issues, and deliver                             
                             additional features.                                                                  
                             To opt out of telemetry, please disable via TelemetryOptOut                           
                             parameter in SDK defaults config. For more information, refer                         
                             to                                                                                    
                             https://sagemaker.readthedocs.io/en/stable/overview.html#confi                        
                             guring-and-using-defaults-with-the-sagemaker-python-sdk.                              

[03/01/25 19:30:04] INFO     Created S3 bucket: sagemaker-us-east-1-715763498984                     ]8;id=392710;file://s:\MLOPS\awssagemaker\venv\lib\site-packages\sagemaker\session.py\session.py]8;;\:]8;id=575443;file://s:\MLOPS\awssagemaker\venv\lib\site-packages\sagemaker\session.py#723\723]8;;\

                    INFO     Creating training-job with name:                                       ]8;id=337218;file://s:\MLOPS\awssagemaker\venv\lib\site-packages\sagemaker\session.py\session.py]8;;\:]8;id=853116;file://s:\MLOPS\awssagemaker\venv\lib\site-packages\sagemaker\session.py#1042\1042]8;;\
                             RF-custom-sklearn-2025-03-01-13-59-58-973                                             

2025-03-01 14:00:08 Starting - Starting the training job......
2025-03-01 14:01:15 Downloading - Downloading input data...
2025-03-01 14:01:41 Downloading - Downloading the training image...
2025-03-01 14:02:32 Training - Training image download completed. Training in progress.
2025-03-01 14:02:32 Uploading - Uploading generated training model.2025-03-01 14:02:25,733 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2025-03-01 14:02:25,737 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2025-03-01 14:02:25,779 sagemaker_sklearn_container.training INFO     Invoking user training script.
2025-03-01 14:02:25,987 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2025-03-01 14:02:25,999 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2025-03-01 14:02:26,011 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2025-03-01 14:0

### To get the model from S3

In [65]:
sklearn_estimator.latest_training_job.wait(logs="None")
artifact = sm_boto3.describe_training_job(
    TrainingJobName=sklearn_estimator.latest_training_job.name
)["ModelArtifacts"]["S3ModelArtifacts"]


2025-03-01 14:02:45 Starting - Preparing the instances for training
2025-03-01 14:02:45 Downloading - Downloading the training image
2025-03-01 14:02:45 Training - Training image download completed. Training in progress.
2025-03-01 14:02:45 Uploading - Uploading generated training model
2025-03-01 14:02:45 Completed - Training job completed


In [66]:
artifact

's3://sagemaker-us-east-1-715763498984/RF-custom-sklearn-2025-03-01-13-59-58-973/output/model.tar.gz'

### Deploy the Model For Endpoint

In [67]:
from sagemaker.sklearn.model import SKLearnModel
from time import gmtime,strftime


model_name="Custom-sklearn-model-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
model=SKLearnModel(
    name=model_name,
    model_data=artifact,
    role="arn:aws:iam::715763498984:role/sagemakeraccess",
    entry_point="script.py",
    framework_version=FRAMEWORK_VERSION
)

In [68]:
model

In [69]:
## Endpoint deployment
endpoint_name="Custom-sklearn-model-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print("EndpointName={}".format(endpoint_name))

predictor=model.deploy(
    initial_instance_count=1,
    instance_type="ml.m4.xlarge",
    endpoint_name=endpoint_name

)

EndpointName=Custom-sklearn-model-2025-03-01-14-09-38


[03/01/25 19:39:45] INFO     Creating model with name: Custom-sklearn-model-2025-03-01-14-09-08     ]8;id=231297;file://s:\MLOPS\awssagemaker\venv\lib\site-packages\sagemaker\session.py\session.py]8;;\:]8;id=408944;file://s:\MLOPS\awssagemaker\venv\lib\site-packages\sagemaker\session.py#4094\4094]8;;\

[03/01/25 19:39:47] INFO     Creating endpoint-config with name                                     ]8;id=321443;file://s:\MLOPS\awssagemaker\venv\lib\site-packages\sagemaker\session.py\session.py]8;;\:]8;id=255974;file://s:\MLOPS\awssagemaker\venv\lib\site-packages\sagemaker\session.py#5889\5889]8;;\
                             Custom-sklearn-model-2025-03-01-14-09-38                                              

[03/01/25 19:39:48] INFO     Creating endpoint with name Custom-sklearn-model-2025-03-01-14-09-38   ]8;id=769982;file://s:\MLOPS\awssagemaker\venv\lib\site-packages\sagemaker\session.py\session.py]8;;\:]8;id=168499;file://s:\MLOPS\awssagemaker\venv\lib\site-packages\sagemaker\session.py#4711\4711]8;;\

------!

In [70]:
predictor

In [71]:
testX[features][0:2]

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,pc,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi
405,1454,1,0.5,1,1,0,34,0.7,83,4,3,250,1033,3419,7,5,5,1,1,0
1190,1092,1,0.5,1,10,0,11,0.5,167,3,14,468,571,737,14,4,11,0,1,0


In [73]:
print(predictor.predict(testX[features][:2].values.tolist()))

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 print(predictor.predict(testX[features][:2].values.tolist()))                                │
│   2                                                                                              │
│                                                                                                  │
│ s:\MLOPS\awssagemaker\venv\lib\site-packages\sagemaker\base_predictor.py:212 in predict          │
│                                                                                                  │
│   209 │   │   if inference_component_name:                                                       │
│   210 │   │   │   request_args["InferenceComponentName"] = inference_component_name              │
│   211 │   │                                                                                      │
│ ❱ 212 │   │   response = self.sagemaker_session.sagemaker_runtime_client.invoke_endpoint(**req   │
│   213 │   │   return self._handle_response(response)                                             │
│   214 │                                                                                          │
│   215 │   def _handle_response(self, response):                                                  │
│                                                                                                  │
│ s:\MLOPS\awssagemaker\venv\lib\site-packages\botocore\client.py:570 in _api_call                 │
│                                                                                                  │
│    567 │   │   │   │   │   f"{py_operation_name}() only accepts keyword arguments."              │
│    568 │   │   │   │   )                                                                         │
│    569 │   │   │   # The "self" in this scope is referring to the BaseClient.                    │
│ ❱  570 │   │   │   return self._make_api_call(operation_name, kwargs)                            │
│    571 │   │                                                                                     │
│    572 │   │   _api_call.__name__ = str(py_operation_name)                                       │
│    573                                                                                           │
│                                                                                                  │
│ s:\MLOPS\awssagemaker\venv\lib\site-packages\botocore\context.py:124 in wrapper                  │
│                                                                                                  │
│   121 │   │   │   with start_as_current_context():                                               │
│   122 │   │   │   │   if hook:                                                                   │
│   123 │   │   │   │   │   hook()                                                                 │
│ ❱ 124 │   │   │   │   return func(*args, **kwargs)                                               │
│   125 │   │                                                                                      │
│   126 │   │   return wrapper                                                                     │
│   127                                                                                            │
│                                                                                                  │
│ s:\MLOPS\awssagemaker\venv\lib\site-packages\botocore\client.py:1031 in _make_api_call           │
│                                                                                                  │
│   1028 │   │   │   │   "Code"                                                                    │
│   1029 │   │   │   )                                       

In [74]:
sm_boto3.delete_endpoint(EndpointName=endpoint_name)

{'ResponseMetadata': {'RequestId': '0e5ac177-14b5-422c-ab4b-27854b2706ae',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '0e5ac177-14b5-422c-ab4b-27854b2706ae',
   'content-type': 'application/x-amz-json-1.1',
   'date': 'Sat, 01 Mar 2025 14:16:29 GMT',
   'content-length': '0'},
  'RetryAttempts': 0}}